In [2]:
import numpy as np 
import random
import matplotlib.pyplot as plt
import scipy.stats as st 
from scipy.stats import norm
from scipy.stats import burr12
from scipy.fft import fft, ifft
import scipy.linalg as linalg
from scipy.optimize import minimize
import collections 
import time
from itertools import chain
import timeit
import os
import pickle
import random
import pandas as pd
from tabulate import tabulate


# Definition of the kernels and basic functions

In [30]:
def kernel_ka1(x,y):
    return (x*y)**0.7

def kernel_ka2(x,y):
    return (x*y)**0.8

def kernel_ka3(x,y):
    return (x*y)**0.9

def kernel_ka4(x,y):
    return (x*y)**0.5

def kernel_ka5(x,y):
    return (x*y)**0.6

def kernel_ka6(x,y):
    return (x*y)**0.95

def kernel_ka7(x,y):
    return (x*y)**0.4

def kernel_ka8(x,y):
    return (x*y)**0.65

def kernel_ka9(x,y):
    return (x*y)**0.6
    
kernel_functions = {
    'kernel_ka1': kernel_ka1,
    'kernel_ka2': kernel_ka2,
    'kernel_ka3': kernel_ka3,
    'kernel_ka4': kernel_ka4,
    'kernel_ka5': kernel_ka5,
    'kernel_ka6': kernel_ka6,
    'kernel_ka7': kernel_ka7,
    'kernel_ka8': kernel_ka8,
    'kernel_ka9': kernel_ka9
}

In [4]:
#function to compute K(x,y) the entries of our kernels and the denominator to use in the index distribution
def kernel_sum(particles, kernel):
    n = len(particles)
    weights = np.zeros((n, n))
    
    kernel = kernel_functions[kernel]
    
    for i in range(n):
        for j in range(n):
            if i != j:
                weights[i, j] = kernel(particles[i], particles[j])
    
    total_weight = np.sum(weights)

    return weights, total_weight

In [5]:
#function to compute K(x,y) the entries of our kernels and the denominator to use in the index distribution
def kernel_sum_triangular(particles, kernel):
    n = len(particles)
    weights = np.zeros((n, n))
    
    kernel = kernel_functions[kernel]
    
    for i in range(n):
        for j in range(i+1, n):
                weights[i, j] = kernel(particles[i], particles[j])
    
    weights += np.triu(weights, 1).T  
    
    total_weight = np.sum(weights)

    return weights, total_weight

In [6]:
#function that calculates the index distribution matrix
def calculate_P_index(particles, kernel):
    weights, total_weight = kernel_sum_triangular(particles, kernel)
    
    P_index = weights / total_weight

    return P_index

In [7]:
#function to sample from the index distribution in a simple way: 
def inverse_sampling(particles, kernel):
    
    U = random.uniform(0, 1)
    
    P_idx = calculate_P_index(particles, kernel)
    
    #find first index s.t. it is bigger than U
    flat_index = np.where(np.cumsum(P_idx) >= U)[0][0]
    
    #get the 2D index from the flat one
    i, j = np.unravel_index(flat_index, P_idx.shape)
        
    return i,j

In [8]:
#function to compute the transition matrix for index i,j for the Markov process described in (3) from the project description
def transition_matrix(i,j, N):
    M = np.eye(N)
    M[i][j]=1
    M[j][j]=0
    
    return M
    

In [9]:
def simulate_coagulation(N, T, X0, kernel):
    kernel_func = kernel_functions[kernel] 
    
    current_time = 0
    jump_count = 0

    Xt = [X0]
    t = [current_time]

    Q_sum_hat = []
    Q_vec_hat = []
    
    while current_time < T:
        
        # Calculate lambda 
        _, total_weight = kernel_sum_triangular(X0, kernel)
        
        Lambda = total_weight/(2*N)
        Q_sum_hat.append(Lambda.copy())
        
        # Generate waiting time Sn
        Sn = st.expon.rvs(scale=1/Lambda)
        
        # Increment current time by the waiting time
        current_time += Sn
        if current_time > T:
            t.append(T)
            break 
        jump_count +=1
        
        # Sample index for collision
        i,j = inverse_sampling(X0, kernel=kernel)

        # Evaluate the value of the kernel at the sampled index
        kernel_value = kernel_func(X0[i], X0[j])/(2*N)
        Q_vec_hat.append(kernel_value)
        
        # Find new vector of sizes based on transition matrix
        M = transition_matrix(i,j,N)
        X0 = M.dot(X0)

        Xt.append(X0.copy()) 
        t.append(current_time)

        if jump_count == N-1:
            break
       
    return Xt, t, Q_vec_hat, Q_sum_hat 

# Functions needed to compute the weight

In [10]:
def entries_of_Q(kernel_o, Xt, N):
    '''
    Function to compute the entries of the transition rate matrix Q of the true kernel.
    Inputs: 
        kernel_o: kernel function of the true process
        Xt: list of vectors of sizes N at each jumping time t
        N: number of particles
    Outputs:
        Q_vec: transition rates of the Markov chain Xt
        Q_sum: sum of the transition rates of the Markov chain Xt
    '''
    # Find indices that have collided, i.e. if difference is positive corresponds to index i
    # if difference is negative corresponds to index j
    initial_kernel = kernel_functions[kernel_o]
    Q_vec = []
    Q_sum = []

    # Loop to calculate Q_sum
    if len(Xt) == len(Xt[0]): # Check if the chain has evolved until the end
        iterations = len(Xt)-1
    else: 
        iterations = len(Xt)

    for i in range(iterations): 
        _, total_weight = kernel_sum_triangular(Xt[i], kernel_o)
        Lambda = total_weight/(2*N)
        Q_sum.append(Lambda)

    # Loop to calculate Q_vec
    for i in range(len(Xt)-1): 
        diff = Xt[i+1] - Xt[i]
        x_i = np.where(diff > 0)[0][0]  
        x_j = np.where(diff < 0)[0][0]
        Q_vec.append(initial_kernel(Xt[i][x_i], Xt[i][x_j])/(2*N))
    
    return Q_vec, Q_sum     

def weightfun(Q_vec, Q_vec_hat, Q_sum, Q_sum_hat, Jt):
    '''
    Inputs:
        Q_vec: true transition rates of the Markov chain along the chain Xt
        Q_vec_hat: transition rates of the importance kernels along the chain Xt
    Outputs:
        likelihood_ratio: product of the weights along the path
        jump_integrator: integral of the weight difference along the path
     '''
    Q_vec = np.array(Q_vec)
    Q_vec_hat = np.array(Q_vec_hat) 
    likelihood_ratio_vec = Q_vec / Q_vec_hat 
    likelihood_ratio = np.prod(likelihood_ratio_vec)

    # Compute the integral of the difference of the weights
    Q_sum = np.array(Q_sum)
    Q_sum_hat = np.array(Q_sum_hat)
    St = np.diff(np.array(Jt)) #the holding times are the time differences between the jumping times
    jump_diff = Q_sum - Q_sum_hat
    jump_integrator = jump_diff * St #element-wise multiplication
    jump_integrator = np.sum(jump_integrator) #integral of piecewise constant function is the sum of the values
    
    weight = likelihood_ratio * np.exp(-jump_integrator)

    return weight

# The Monte Carlo estimators

In [11]:
def crude_tg(N, T, kernel_o, R, X0):
    '''This function computes the crude monte carlo estimator of tg'''
    samples = []
    for _ in range(R):
        Xt, _, _, _ = simulate_coagulation(N, T, X0,kernel_o)
        G_n = np.max(Xt[-1]) / N 
        samples.append(G_n)
    
    samples = np.array(samples)
    samples = np.where(samples >= 0.5, 1, 0) #if G_n is bigger than 0.5, we set it to 1; otherwise, to 0
    expectation = np.mean(samples)
    variance = np.var(samples)

    return expectation, variance

def importance_sampling_tg(N, T, importance_kernel, kernel_o, R, X0):
    samples = []
    weights = []

    for _ in range(R):
        Xt, Jt, Q_vec_hat, Q_sum_hat = simulate_coagulation(N, T, X0, importance_kernel)
        Q_vec, Q_sum = entries_of_Q(kernel_o, Xt, N)
        weight = weightfun(Q_vec, Q_vec_hat, Q_sum, Q_sum_hat, Jt)
        weights.append(weight)
        G_n = np.max(Xt[-1]) / N
        samples.append(G_n)
        
    # If G_n is bigger than 0.5, we multiply the weight by 1; otherwise, we multiply it by 0    
    weights = np.array(weights)
    samples = np.array(samples)
    weighted_samples = weights * (samples >= 0.5)

    expectation = np.mean(weighted_samples)
    variance = np.var(weighted_samples) 

    return expectation, variance

In [12]:
def crude_c(N, T, kernel_o, R, X0, k_vec):
    '''This function computes the crude monte carlo estimator of c_k at time T'''
    samples = []

    for _ in range(R):
        Xt, _, _, _ = simulate_coagulation(N, T, X0,kernel_o)
        sim = []
        for k in k_vec:
            c_k = np.count_nonzero(Xt[-1] == k) / N
            sim.append(c_k)
        samples.append(sim)

    # Compute the expectation and variance of the estimator; note we have lists in lists
    samples = np.array(samples)
    expectation = np.mean(samples, axis=0)
    variance = np.var(samples, axis=0)

    return expectation, variance

def importance_sampling_c(N, T, importance_kernel, kernel_o, R, X0, k_vec):
    '''This function computes the importance sampling estimator of c_k at time T'''
    samples = []
    weights = []

    for _ in range(R):
        Xt, Jt, Q_vec_hat, Q_sum_hat = simulate_coagulation(N, T, X0, importance_kernel)
        Q_vec, Q_sum = entries_of_Q(kernel_o, Xt, N)
        weight = weightfun(Q_vec, Q_vec_hat, Q_sum, Q_sum_hat, Jt)
        weights.append(weight)
        sim = []
        for k in k_vec:
            c_k = np.count_nonzero(Xt[-1] == k) / N
            sim.append(c_k)
        samples.append(sim)

    # Compute the expectation and variance of the estimator; note we have lists in lists
    weights = np.array(weights)
    samples = np.array(samples)
    weighted_samples = [[w_i*s_i for s_i in sub_samples] for w_i, sub_samples in zip(weights, samples)]

    expectation = np.mean(weighted_samples, axis=0)
    variance = np.var(weighted_samples, axis=0)

    return expectation, variance

# The main to call the simulations

In [28]:
'''Main for importance sampling of tg using kernel ka1/ka2 and finding a good value for b'''
# Set the seed value
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
N = 100
T = 2
importance_kernel = ['kernel_ka2','kernel_ka3', 'kernel_ka6']
b = [0.8, 0.9, 0.95]
kernel_o = 'kernel_ka2'
R = 500
results_tg_b_2 = []
X0 = np.ones(N)
expectation_crude, variance_crude = crude_tg(N, T, kernel_o, R, X0)

for importance_kernel,b in zip(importance_kernel,b):
    start = timeit.default_timer()
    expectation_imp, variance_imp = importance_sampling_tg(N, T, importance_kernel, kernel_o, R, X0)
    stop = timeit.default_timer()
    print("time", stop-start)
    results_tg_b_2.append((N, R, b, expectation_crude, variance_crude, expectation_imp, variance_imp))

# Save results to CSV files
np.savetxt('results_tg_b_2.csv', results_tg_b_2, delimiter=',', header='N,R,b,mu_crude,var_crude,mu_imp,var_imp', comments='')

time 496.1429606999736
time 538.475357999996
time 600.6869149999984


In [36]:
'''Main for importance sampling of tg using kernel ka1 and ka3'''
# Set the seed value
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
N_val = [100,200,500]
T = 2
importance_kernel = 'kernel_ka3'
kernel_o = 'kernel_ka1'
R = [500, 100, 100]
results_tg = []

for N,R in zip(N_val, R):
    print(f"N = {N}, R = {R}")
    start = timeit.default_timer()
    X0 = np.ones(N)
    expectation_crude, variance_crude = crude_tg(N, T, kernel_o, R, X0)
    expectation_imp, variance_imp = importance_sampling_tg(N, T, importance_kernel, kernel_o, R, X0)
    stop = timeit.default_timer()
    print("time", stop-start)
    results_tg.append((N, R, expectation_crude, variance_crude, expectation_imp, variance_imp))

# Save results to CSV files
np.savetxt('results_tg.csv', results_tg, delimiter=',', header='N,R,mu_crude,var_crude,mu_imp,var_imp', comments='')

N = 100, R = 500
time 533.5149435999992
N = 200, R = 100
time 936.2190885000018
N = 500, R = 100
time 12241.264285700003


In [33]:
'''Main for importance sampling of tg using kernel ka2 and ka4'''
# Set the seed value
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
N_val = [100,200,500]
T = 2
importance_kernel = 'kernel_ka3'
kernel_o = 'kernel_ka2'
R = [500, 100, 10]
results_tg_2_2 = []

for N,R in zip(N_val, R):
    print(f"N = {N}, R = {R}")
    start = timeit.default_timer()
    X0 = np.ones(N)
    expectation_crude, variance_crude = crude_tg(N, T, kernel_o, R, X0)
    expectation_imp, variance_imp = importance_sampling_tg(N, T, importance_kernel, kernel_o, R, X0)
    stop = timeit.default_timer()
    print("time", stop-start)
    results_tg_2_2.append((N, R, expectation_crude, variance_crude, expectation_imp, variance_imp))

# Save results to CSV files
np.savetxt('results_tg_2_2.csv', results_tg_2_2, delimiter=',', header='N,R,mu_crude,var_crude,mu_imp,var_imp', comments='')

N = 100, R = 500
time 516.9946870999993
N = 200, R = 100
time 1043.1124371000042
N = 500, R = 10
time 1942.4413836000022


In [32]:
'''Main for importance sampling of c_k; here we try to find a good kernel for ka1'''
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
N_val = [100]
t_val = [1,2,3,4,5]
importance_kernel_list = ['kernel_ka7', 'kernel_ka4', 'kernel_ka9', 'kernel_ka8']
b_list = [0.4, 0.5, 0.6, 0.65]
kernel_o = 'kernel_ka1'
R=[100]
k_vec = [1,2,3,5]
results_c_full = []

for N,R in zip(N_val, R):
    print(f"N = {N}, R = {R}")
    start = timeit.default_timer()
    X0 = np.ones(N)
    for t in t_val:
        expectation_crude_c, variance_crude_c = crude_c(N, t, kernel_o, R, X0, k_vec) 
        for importance_kernel, b in zip(importance_kernel_list, b_list):
            expectation_imp_c, variance_imp_c = importance_sampling_c(N, t, importance_kernel, kernel_o, R, X0, k_vec)
            for i in range(len(k_vec)):
                results_c_full.append((N, b, t, R, k_vec[i], expectation_crude_c[i], variance_crude_c[i], 
                                expectation_imp_c[i], variance_imp_c[i]))
    stop = timeit.default_timer()
    print("time", stop-start)

# Save results to CSV files
np.savetxt('results_c_full.csv', results_c_full, delimiter=',', header='N,b,T,R,k,mu_crude,var_crude,mu_imp,var_imp', comments='')

N = 100, R = 100
time 1921.0133901999798


In [ ]:
'''Main for importance sampling of c_k; here we try to find a good kernel for ka2'''
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
N_val = [100]
t_val = [1,2,3,4,5]
importance_kernel_list = ['kernel_ka4', 'kernel_ka9', 'kernel_ka8', 'kernel_ka1']
b_list = [0.5, 0.6, 0.65, 0.7]
kernel_o = 'kernel_ka2'
R=[100]
k_vec = [1,2,3,5]
results_c_full_2 = []

for N,R in zip(N_val, R):
    print(f"N = {N}, R = {R}")
    start = timeit.default_timer()
    X0 = np.ones(N)
    for t in t_val:
        expectation_crude_c, variance_crude_c = crude_c(N, t, kernel_o, R, X0, k_vec) 
        for importance_kernel, b in zip(importance_kernel_list, b_list):
            expectation_imp_c, variance_imp_c = importance_sampling_c(N, t, importance_kernel, kernel_o, R, X0, k_vec)

            for i in range(len(k_vec)):
                results_c_full_2.append((N, b, t, R, k_vec[i], expectation_crude_c[i], variance_crude_c[i], 
                                expectation_imp_c[i], variance_imp_c[i]))
    stop = timeit.default_timer()
    print("time", stop-start)

# Save results to CSV files
np.savetxt('results_c_full_2.csv', results_c_full_2, delimiter=',', header='N,b,T,R,k,mu_crude,var_crude,mu_imp,var_imp', comments='')

# Create tables that can be used for latex

In [29]:
# Load the CSV file
results_tg = np.loadtxt('results_tg_b_2.csv', delimiter=',', skiprows=1)

# Add a title to the table
table_title = r'$K(x,y) = (xy)^{0.7} \, , \, \hat{K}(x,y) = (xy)^{b}$'

# Print the results in a LaTeX table
table_headers = ['$b$', '$\\mu_{\\text{CR}}$', '$\\sigma_{\\text{CR}}$', '$\\mu_{\\text{IS}}$', '$\\sigma_{\\text{IS}}$']
table_data = []

for result in results_tg:
    N, R, b, expectation_crude, variance_crude, expectation_imp, variance_imp = result
    table_data.append([b, f'{expectation_crude:.4f}', f'{np.sqrt(variance_crude):.4f}', f'{expectation_imp:.4f}', f'{np.sqrt(variance_imp):.4f}'])

# Combine name, description, title, and table
latex_table = table_title + '\n\n' + tabulate(table_data, headers=table_headers, tablefmt='latex_raw')
print(latex_table)

# Save LaTeX table to a file
with open('results_tg_b_2_table.tex', 'w') as f:
    f.write(latex_table)

$K(x,y) = (xy)^{0.7} \, , \, \hat{K}(x,y) = (xy)^{b}$

\begin{tabular}{rrrrr}
\hline
   $b$ &   $\mu_{\text{CR}}$ &   $\sigma_{\text{CR}}$ &   $\mu_{\text{IS}}$ &   $\sigma_{\text{IS}}$ \\
\hline
  0.8  &               0.254 &                 0.4353 &              0.252  &                 0.4342 \\
  0.9  &               0.264 &                 0.4408 &              0.2379 &                 0.4704 \\
  0.95 &               0.222 &                 0.4156 &              0.2622 &                 1.3502 \\
\hline
\end{tabular}


In [61]:
# Load the CSV file
results_tg = np.loadtxt('results_tg.csv', delimiter=',', skiprows=1)

# Add a title to the table
table_title = r'$K(x,y) = (xy)^{0.7} \, , \, \hat{K}(x,y) = (xy)^{0.9}$'

# Print the results in a LaTeX table
table_headers = ['$N$', '$R$', '$\\mu_{\\text{CR}}$', '$\\sigma_{\\text{CR}}$', '$\\mu_{\\text{IS}}$', '$\\sigma_{\\text{IS}}$']
table_data = []

for result in results_tg:
    N, R, expectation_crude, variance_crude, expectation_imp, variance_imp = result
    table_data.append([int(N), int(R), f'{expectation_crude:.4f}', f'{np.sqrt(variance_crude):.4f}', f'{expectation_imp:.4f}', f'{np.sqrt(variance_imp):.4f}'])

# Combine name, description, title, and table
latex_table = table_title + '\n\n' + tabulate(table_data, headers=table_headers, tablefmt='latex_raw')
print(latex_table)

# Save LaTeX table to a file
with open('results_tg_table.tex', 'w') as f:
    f.write(latex_table)

$K(x,y) = (xy)^{0.7} \, , \, \hat{K}(x,y) = (xy)^{0.9}$

\begin{tabular}{rrrrrr}
\hline
   $N$ &   $R$ &   $\mu_{\text{CR}}$ &   $\sigma_{\text{CR}}$ &   $\mu_{\text{IS}}$ &   $\sigma_{\text{IS}}$ \\
\hline
   100 &   500 &               0.028 &                  0.165 &              0.012  &                 0.0441 \\
   200 &   100 &               0     &                  0     &              0.0001 &                 0.0005 \\
   500 &   100 &               0     &                  0     &              0      &                 0      \\
\hline
\end{tabular}


In [34]:
# Load the CSV file
results_tg = np.loadtxt('results_tg_2_2.csv', delimiter=',', skiprows=1)

# Add a title to the table
table_title = r'$K(x,y) = (xy)^{0.8} \, , \, \hat{K}(x,y) = (xy)^{0.9}$'

# Print the results in a LaTeX table
table_headers = ['$N$', '$R$', '$\\mu_{\\text{CR}}$', '$\\sigma_{\\text{CR}}$', '$\\mu_{\\text{IS}}$', '$\\sigma_{\\text{IS}}$']
table_data = []

for result in results_tg:
    N, R, expectation_crude, variance_crude, expectation_imp, variance_imp = result
    table_data.append([int(N), int(R), f'{expectation_crude:.4f}', f'{np.sqrt(variance_crude):.4f}', f'{expectation_imp:.4f}', f'{np.sqrt(variance_imp):.4f}'])

# Combine name, description, title, and table
latex_table = table_title + '\n\n' + tabulate(table_data, headers=table_headers, tablefmt='latex_raw')
print(latex_table)

# Save LaTeX table to a file
with open('results_tg_table_2_2.tex', 'w') as f:
    f.write(latex_table)


$K(x,y) = (xy)^{0.8} \, , \, \hat{K}(x,y) = (xy)^{0.9}$

\begin{tabular}{rrrrrr}
\hline
   $N$ &   $R$ &   $\mu_{\text{CR}}$ &   $\sigma_{\text{CR}}$ &   $\mu_{\text{IS}}$ &   $\sigma_{\text{IS}}$ \\
\hline
   100 &   500 &               0.254 &                 0.4353 &              0.2192 &                 0.4713 \\
   200 &   100 &               0.1   &                 0.3    &              0.1186 &                 0.3956 \\
   500 &    10 &               0     &                 0      &              0.0109 &                 0.0315 \\
\hline
\end{tabular}


In [44]:
# Load data
results_c = np.loadtxt('results_c_full.csv', delimiter=',', skiprows=1)

# Add a title to the table
table_title = r'$K(x,y) = (xy)^{0.7} \, , \, \hat{K}(x,y) = (xy)^{b}$'

# Print the results in a LaTeX table
table_headers = ['$b$', '$k$', '$T$','$\\mu_{\\text{CR}}$', '$\\sigma_{\\text{CR}}$', '$\\mu_{\\text{IS}}$', '$\\sigma_{\\text{IS}}$']
table_data = []

# Keep track of 'a' value
a_value = None

for i, result in enumerate(results_c):
    _, b, T, _, k, expectation_crude_c, variance_crude_c, expectation_imp_c, variance_imp_c = result
    
    # Only display 'a' value once every four rows
    if i % 4 == 0:
        a_value = b
    
    # Append data without 'b'
    table_data.append([b if i % 4 == 0 else '', int(k), int(T), f'{expectation_crude_c:.4f}', f'{np.sqrt(variance_crude_c):.4f}', f'{expectation_imp_c:.4f}', f'{np.sqrt(variance_imp_c):.4f}'])
    
    # Add a horizontal line every four rows
    if (i + 1) % 4 == 0 and i + 1 != len(results_c):
        table_data.append(['-----'] * len(table_headers))  # Horizontal line

# Combine name, description, title, and table
latex_table = table_title + '\n\n' + tabulate(table_data, headers=table_headers, tablefmt='latex_raw')
print(latex_table)

# Save LaTeX table to a file
with open('results_c_full_table.tex', 'w') as f:
    f.write(latex_table)


$K(x,y) = (xy)^{0.7} \, , \, \hat{K}(x,y) = (xy)^{b}$

\begin{tabular}{lllllll}
\hline
 $b$   & $k$   & $T$   & $\mu_{\text{CR}}$   & $\sigma_{\text{CR}}$   & $\mu_{\text{IS}}$   & $\sigma_{\text{IS}}$   \\
\hline
 0.4   & 1     & 2     & 0.4014              & 0.0561                 & 0.3076              & 0.5012                 \\
       & 2     & 2     & 0.0968              & 0.0312                 & 0.0790              & 0.1422                 \\
       & 3     & 2     & 0.0415              & 0.0176                 & 0.0361              & 0.0521                 \\
       & 5     & 2     & 0.0111              & 0.0094                 & 0.0079              & 0.0174                 \\
 ----- & ----- & ----- & -----               & -----                  & -----               & -----                  \\
 0.5   & 1     & 2     & 0.3992              & 0.0549                 & 0.5769              & 0.8872                 \\
       & 2     & 2     & 0.1009              & 0.0281             

In [41]:
# Load data
results_c = np.loadtxt('results_c_full_2.csv', delimiter=',', skiprows=1)

# Add a title to the table
table_title = r'$K(x,y) = (xy)^{0.8} \, , \, \hat{K}(x,y) = (xy)^{b}$'

# Print the results in a LaTeX table
table_headers = ['$b$', '$k$', '$T$','$\\mu_{\\text{CR}}$', '$\\sigma_{\\text{CR}}$', '$\\mu_{\\text{IS}}$', '$\\sigma_{\\text{IS}}$']
table_data = []

# Keep track of 'a' value
a_value = None

for i, result in enumerate(results_c):
    _, b, T, _, k, expectation_crude_c, variance_crude_c, expectation_imp_c, variance_imp_c = result
    
    # Only display 'a' value once every four rows
    if i % 4 == 0:
        a_value = b
    
    # Append data without 'b'
    table_data.append([b if i % 4 == 0 else '', int(k), int(T), f'{expectation_crude_c:.4f}', f'{np.sqrt(variance_crude_c):.4f}', f'{expectation_imp_c:.4f}', f'{np.sqrt(variance_imp_c):.4f}'])
    
    # Add a horizontal line every four rows
    if (i + 1) % 4 == 0 and i + 1 != len(results_c):
        table_data.append(['-----'] * len(table_headers))  # Horizontal line

# Combine name, description, title, and table
latex_table = table_title + '\n\n' + tabulate(table_data, headers=table_headers, tablefmt='latex_raw')
print(latex_table)

# Save LaTeX table to a file
with open('results_c_full_2_table.tex', 'w') as f:
    f.write(latex_table)


$K(x,y) = (xy)^{0.8} \, , \, \hat{K}(x,y) = (xy)^{b}$

\begin{tabular}{lllllll}
\hline
 $b$   & $k$   & $T$   & $\mu_{\text{CR}}$   & $\sigma_{\text{CR}}$   & $\mu_{\text{IS}}$   & $\sigma_{\text{IS}}$   \\
\hline
 0.5   & 1     & 1     & 0.3891              & 0.0592                 & 0.4652              & 1.5824                 \\
       & 2     & 1     & 0.0875              & 0.0260                 & 0.1023              & 0.3268                 \\
       & 3     & 1     & 0.0372              & 0.0161                 & 0.0387              & 0.1387                 \\
       & 5     & 1     & 0.0115              & 0.0113                 & 0.0197              & 0.0929                 \\
 ----- & ----- & ----- & -----               & -----                  & -----               & -----                  \\
 0.6   & 1     & 1     & 0.3976              & 0.0577                 & 0.4021              & 0.4462                 \\
       & 2     & 1     & 0.0852              & 0.0268             

In [60]:
# Load data
results_c = np.loadtxt('results_c_final_2_3.csv', delimiter=',', skiprows=1)

# Add a title to the table
table_title = r'$K(x,y) = (xy)^{0.8} \, , \, \hat{K}(x,y) = (xy)^{0.6}$'

# Print the results in a LaTeX table
table_headers = ['$T$', '$N$','$R$','$k$', '$\\mu_{\\text{CR}}$', '$\\sigma_{\\text{CR}}$', '$\\mu_{\\text{IS}}$', '$\\sigma_{\\text{IS}}$']
table_data = []

# Keep track of 'a' value
a_value = None

for i, result in enumerate(results_c):
    N, _, T, R, k, expectation_crude_c, variance_crude_c, expectation_imp_c, variance_imp_c = result
       
    # Append data without 'b'
    table_data.append([T,int(N), int(R), int(k), f'{expectation_crude_c:.4f}', f'{np.sqrt(variance_crude_c):.4f}', f'{expectation_imp_c:.4f}', f'{np.sqrt(variance_imp_c):.4f}'])
    table_data.append(['-----'] * len(table_headers))

# Combine name, description, title, and table
latex_table = table_title + '\n\n' + tabulate(table_data, headers=table_headers, tablefmt='latex_raw')
print(latex_table)

# Save LaTeX table to a file
with open('results_c_final_2_3.tex', 'w') as f:
    f.write(latex_table)


$K(x,y) = (xy)^{0.8} \, , \, \hat{K}(x,y) = (xy)^{0.6}$

\begin{tabular}{llllllll}
\hline
 $T$   & $N$   & $R$   & $k$   & $\mu_{\text{CR}}$   & $\sigma_{\text{CR}}$   & $\mu_{\text{IS}}$   & $\sigma_{\text{IS}}$   \\
\hline
 0.6   & 100   & 100   & 1     & 0.5699              & 0.0723                 & 0.6051              & 0.4870                 \\
 ----- & ----- & ----- & ----- & -----               & -----                  & -----               & -----                  \\
 0.6   & 100   & 100   & 2     & 0.1034              & 0.0301                 & 0.1078              & 0.0713                 \\
 ----- & ----- & ----- & ----- & -----               & -----                  & -----               & -----                  \\
 0.6   & 100   & 100   & 3     & 0.0305              & 0.0197                 & 0.0384              & 0.0407                 \\
 ----- & ----- & ----- & ----- & -----               & -----                  & -----               & -----                  \\
 0.6   